In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Load the KITTI or Waymo dataset (Here, we'll assume a CSV file with annotations for simplicity)
# Replace with actual file paths of your dataset (KITTI/Waymo)

dataset_path = 'path_to_kitti_or_waymo.csv'  # Example path, update accordingly
data = pd.read_csv(dataset_path)

# Display the first 10 samples of the dataset
print("First 10 samples of the dataset:")
print(data.head(10))

In [ ]:
import torch
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor

# Load YOLO model for object detection (Pre-trained)
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Example YOLOv5 model

# Example model for trajectory prediction (LSTM)
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Example Random Forest, XGBoost, and SVM models for hazard prediction
def build_rf_model():
    return RandomForestRegressor(n_estimators=100, random_state=42)

def build_xgb_model():
    return XGBRegressor(n_estimators=100, random_state=42)

def build_svm_model():
    return SVR(kernel='rbf', C=1e3, gamma=0.1)

# Instantiate models
lstm_model = build_lstm_model(input_shape=(10, 1))  # Example input shape
rf_model = build_rf_model()
xgb_model = build_xgb_model()
svm_model = build_svm_model()

# Train models with preprocessed data (this assumes your features and labels are ready)
X_train, X_test, y_train, y_test = train_test_split(data.drop('label', axis=1), data['label'], test_size=0.2, random_state=42)

# Fit all models
lstm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Predictions
lstm_preds = lstm_model.predict(X_test)
rf_preds = rf_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)
svm_preds = svm_model.predict(X_test)

# Evaluate Models
mse_lstm = mean_squared_error(y_test, lstm_preds)
r2_lstm = r2_score(y_test, lstm_preds)

mse_rf = mean_squared_error(y_test, rf_preds)
r2_rf = r2_score(y_test, rf_preds)

mse_xgb = mean_squared_error(y_test, xgb_preds)
r2_xgb = r2_score(y_test, xgb_preds)

mse_svm = mean_squared_error(y_test, svm_preds)
r2_svm = r2_score(y_test, svm_preds)

# Display metrics
print(f"LSTM MSE: {mse_lstm}, R2: {r2_lstm}")
print(f"Random Forest MSE: {mse_rf}, R2: {r2_rf}")
print(f"XGBoost MSE: {mse_xgb}, R2: {r2_xgb}")
print(f"SVM MSE: {mse_svm}, R2: {r2_svm}")

# Weighted Ensemble Model
ensemble_model = VotingRegressor(estimators=[('lstm', lstm_model), ('rf', rf_model), ('xgb', xgb_model), ('svm', svm_model)])
ensemble_model.fit(X_train, y_train)

# Predict with ensemble model
ensemble_preds = ensemble_model.predict(X_test)

# Ensemble evaluation
mse_ensemble = mean_squared_error(y_test, ensemble_preds)
r2_ensemble = r2_score(y_test, ensemble_preds)

print(f"Ensemble Model MSE: {mse_ensemble}, R2: {r2_ensemble}")


In [ ]:
# Plotting Actual vs Predicted for all models
plt.figure(figsize=(15, 10))

# Actual vs Predicted for LSTM
plt.subplot(231)
plt.scatter(y_test, lstm_preds, color='red')
plt.title('LSTM Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

# Actual vs Predicted for Random Forest
plt.subplot(232)
plt.scatter(y_test, rf_preds, color='green')
plt.title('Random Forest Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

# Actual vs Predicted for XGBoost
plt.subplot(233)
plt.scatter(y_test, xgb_preds, color='blue')
plt.title('XGBoost Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

# Actual vs Predicted for SVM
plt.subplot(234)
plt.scatter(y_test, svm_preds, color='purple')
plt.title('SVM Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

# Actual vs Predicted for Ensemble Model
plt.subplot(235)
plt.scatter(y_test, ensemble_preds, color='orange')
plt.title('Ensemble Model Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

# Show plots
plt.tight_layout()
plt.show()
